In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import urllib.request
import urllib.parse
import pandas as pd
import requests


In [11]:
players = ['hide on bush', 'T1 Ellim', '칸 나', '롤오브레전드1234', 'T1 Gumayusi']
dicts = []
for player in players:
    url = f'https://www.op.gg/summoner/userName={player}'
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
    req = requests.get(url, headers = header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    player_id = []
    results = []; play_times = []; game_types = []; champ_names = []; kills = [] 
    deaths = []; assists = []; kdas = []; levels = []; css = []; ck_rates = []
    board_list = soup.select('.GameItemWrap')
    for board in tqdm_notebook(board_list):
        if player == 'hide on bush':
            player_id.append('Faker')
        elif player == 'T1 Ellim':
            player_id.append('Ellim')
        elif player == '칸 나':
            player_id.append('Canna')
        elif player == '롤오브레전드1234':
            player_id.append('Effort')
        elif player == 'T1 Gumayusi':
            player_id.append('Gumayusi')
        result = board.select_one('.GameResult').string.strip()
        play_time = board.select_one('.GameLength').string.strip()
        game_type = board.select_one('.GameType').string.strip()
        champion = board.select_one('.ChampionName')
        champ_name = champion.a.string
        kill = int(board.select_one('.Kill').string.strip())
        death = int(board.select_one('.Death').string.strip())
        assist = int(board.select_one('.Assist').string.strip())
        if death != 0:
            kda = round((kill + assist) / death, 2)
        else:
            kda = 'Perfect'
        level = board.select_one('.Level').string.strip()[5:]
        cs = board.select_one('.CS').span.string.split()[0]
        ck_rate = board.select_one('.CKRate').string.strip().split()[1]

        results.append(result); play_times.append(play_time); game_types.append(game_type); champ_names.append(champ_name)
        kills.append(kill); deaths.append(death); assists.append(assist); kdas.append(kda); levels.append(level)
        css.append(cs); ck_rates.append(ck_rate)

    mat_his = {
        'Player': player_id,
        'Result': results,
        'Game Length': play_times,
        'Game Type': game_types,
        'Champion': champ_names,
        'Kill': kills,
        'Death': deaths,
        'Assist': assists,
        'KDA': kdas,
        'Level': levels,
        'CS': css,
        'CK Rate': ck_rates
    }
    dicts.append(mat_his)

In [12]:
dfs = []
for i in range(len(dicts)):
    df = pd.DataFrame(dicts[i])
    dfs.append(df)

In [18]:
Faker = dfs[0].set_index('Player'); Ellim = dfs[1].set_index('Player'); Canna = dfs[2].set_index('Player'); Effort = dfs[3].set_index('Player'); Gumayusi = dfs[4].set_index('Player')
tmp = Faker.append(Ellim)
tmp = tmp.append(Canna)
tmp = tmp.append(Effort)
T1 = tmp.append(Gumayusi)

In [20]:
T1.to_csv('T1 Match History.csv', encoding='utf8')